In [1]:
from datasets import load_dataset, concatenate_datasets, Dataset
from jinja2 import Template
from vllm import LLM, SamplingParams

import json

In [2]:
NUM_OUTPUTS = 1024 * 10
NUM_GPUS = 4
BATCH_SIZE = 256
PUSH_INTERVAL = 8

In [3]:
system_prompt = """
Your goal is to create high quality learning content, like Wikipedia articles, that provides information collected through experience and research."
""".strip()

In [4]:
model_id = "meta-llama/Llama-3.1-405B-Instruct-FP8"

In [5]:
llm = LLM(model=model_id, max_model_len=4096, tensor_parallel_size=NUM_GPUS, gpu_memory_utilization=0.98)

INFO 12-26 20:45:32 config.py:478] This model supports multiple tasks: {'classify', 'generate', 'reward', 'score', 'embed'}. Defaulting to 'generate'.
INFO 12-26 20:45:32 config.py:1216] Defaulting to use mp for distributed inference
INFO 12-26 20:45:32 llm_engine.py:249] Initializing an LLM engine (v0.6.5) with config: model='meta-llama/Llama-3.1-405B-Instruct-FP8', speculative_config=None, tokenizer='meta-llama/Llama-3.1-405B-Instruct-FP8', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=fbgemm_fp8, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None

Loading safetensors checkpoint shards:   0% Completed | 0/109 [00:00<?, ?it/s]


(VllmWorkerProcess pid=52322) INFO 12-26 20:47:38 model_runner.py:1097] Loading model weights took 113.4847 GB
INFO 12-26 20:47:38 model_runner.py:1097] Loading model weights took 113.4847 GB
(VllmWorkerProcess pid=52321) INFO 12-26 20:47:38 model_runner.py:1097] Loading model weights took 113.4847 GB
(VllmWorkerProcess pid=52320) INFO 12-26 20:47:38 model_runner.py:1097] Loading model weights took 113.4847 GB
(VllmWorkerProcess pid=52321) (VllmWorkerProcess pid=52320) INFO 12-26 20:47:42 worker.py:241] Memory profiling takes 3.74 seconds
INFO 12-26 20:47:42 worker.py:241] Memory profiling takes 3.74 seconds
(VllmWorkerProcess pid=52321) (VllmWorkerProcess pid=52320) INFO 12-26 20:47:42 worker.py:241] the current vLLM instance can use total_gpu_memory (139.72GiB) x gpu_memory_utilization (0.98) = 136.92GiB
INFO 12-26 20:47:42 worker.py:241] the current vLLM instance can use total_gpu_memory (139.72GiB) x gpu_memory_utilization (0.98) = 136.92GiB
(VllmWorkerProcess pid=52321) (VllmWorke

In [7]:
def generate_content(n):
    message = [{"role": "system", "content": system_prompt},
               {"role": "user", "content": "Write a long article.\n\nTitle:"}]

    outputs = llm.chat([message]*n, SamplingParams(temperature=1, top_p=0.9, max_tokens=3584))

    return {"synthetic_content": [output.outputs[0].text.strip() for output in outputs]}

In [8]:
def generate_content_base(n):
    prompt = "Here is long high quality learning article, like a Wikipedia article, that provides information collected through experience and research. The article is formatted for easy readibility.\n\nTitle:"
    outputs = llm.generate([prompt]*n, SamplingParams(temperature=0.75, top_p=0.9, max_tokens=4096))

    return {"synthetic_content": [output.outputs[0].text.strip() for output in outputs]}

In [9]:
ds = Dataset.from_dict(generate_content(4))

INFO 12-26 20:50:51 chat_utils.py:333] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.


Processed prompts: 100% 4/4 [00:57<00:00, 14.28s/it, est. speed input: 4.62 toks/s, output: 82.00 toks/s]


In [17]:
ds_id = "amang1802/synthetic_data_unconditioned_L3.1_405B_Instruct"

In [18]:
num_batches = NUM_OUTPUTS // BATCH_SIZE
for i in range(num_batches):
    new_data = Dataset.from_dict(generate_content_base(BATCH_SIZE))
    ds = concatenate_datasets([ds, new_data])
    print(f"Dataset size: {ds.num_rows}")
    if (i+1) % PUSH_INTERVAL == 0:
        ds.push_to_hub(ds_id)

Processed prompts:   1% 3/256 [00:54<54:35, 12.95s/it, est. speed input: 1.86 toks/s, output: 27.11 toks/s]  

WARNING 12-26 20:54:06 scheduler.py:1555] Sequence group 259 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1


Processed prompts: 100% 256/256 [03:58<00:00,  1.07it/s, est. speed input: 36.50 toks/s, output: 893.19 toks/s] 


Dataset size: 260


Processed prompts:   3% 8/256 [00:58<08:54,  2.16s/it, est. speed input: 4.67 toks/s, output: 72.40 toks/s]  

...

Dataset size: 10244


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/290 [00:00<?, ?B/s]